In [9]:
import numpy as np
import pandas as pd

import re
import gensim

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

# from gensim.models import Word2Vec
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Embedding
# from keras_preprocessing.sequence import pad_sequences
# from sklearn.model_selection import train_test_split

In [10]:
import nltk

class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [11]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # kata kosong akan me-return vektor 0 dengan dimensi yang sama untuk semua vektor lainnya
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    def cek(self,xx):
      return self.word2vec.wv[xx]
    def fit_transform(self, X, y=None):
        return self.transform(X)

### Load pre-trained word2vec model

In [14]:
w2v_model_100 = gensim.models.KeyedVectors.load_word2vec_format("../MODEL/idwiki_word2vec_100/idwiki_word2vec_100.model")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v_model_100)

### Load dataset

In [ ]:
df = pd.read_csv("../DATA/cleaned_data_non-numeric_review.csv")
df

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
df.isnull().sum()

_`total label value`_

In [ ]:
df['label'].value_counts()

In [ ]:
# define a function to generate document embeddings
def generate_doc_embedding(doc):
    # tokenize the document
    tokens = doc.split()
    # remove out-of-vocabulary words
    tokens = [token for token in tokens if token in w2v_model.wv.index_to_key]
    # generate word embeddings for each token
    embeddings = [w2v_model.wv[token] for token in tokens]
    # combine the embeddings into a single vector
    if len(embeddings) > 0:
        doc_embedding = np.mean(embeddings, axis=0)
    else:
        doc_embedding = np.zeros((w2v_model.vector_size,))
    return doc_embedding


In [ ]:
texts = df.drop(['stemming'], axis = 1) 
label = df['label']

texts

In [ ]:
texts = df['content'].values
label = df['label'].values

In [ ]:
texts[1]

In [ ]:
label[1]

In [ ]:

def merge(list1, list2):
     
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

data = merge(texts, label)
data

In [ ]:
data_train = data[:224]
data_test = data[224:]


In [ ]:
data_train

In [ ]:
# Convert text labels to numeric labels if necessary
# label_to_num = {"negative": -1, "netral": 0, "positif": 1}
# labels = np.array([label_to_num[label] for label in label])
# labels

In [ ]:
# Reshape the input data to be 3-dimensional (samples, time steps, features)
# X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
# list_text = list(texts.itertuples(index=False, name=None))
# list_text

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
# generate training and testing embeddings
X_train = [generate_doc_embedding(doc) for doc, label in data_train]
y_train = [label for doc, label in data_train]
X_test = [generate_doc_embedding(doc) for doc, label in data_test]
y_test = [label for doc, label in data_test]

X_train

In [ ]:
# calculate word2vec feature weights
weights = w2v_model.wv.vectors.mean(axis=0)

In [ ]:
# apply feature weighting to the embeddings
X_train = np.multiply(X_train, weights)
X_test = np.multiply(X_test, weights)

In [ ]:
num_sample, input_dim = X_train.shape
num_classes = X_test.shape[1]
X_train = np.reshape(X_train, (num_sample, 1, input_dim))
# data = np.array(data)
# data = np.zeros((100, 2))
# data


In [ ]:
X_train

In [ ]:
X_train = np.zeros((32, input_dim))


In [ ]:
# define and train the RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(1, input_dim), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)